# Retrieve input for experimental design

## Import libraries and files

In [1]:
from openai import OpenAI
import openai
import os
import json
import requests
import nltk
from textblob import TextBlob
from textblob.sentiments import NaiveBayesAnalyzer


client = OpenAI()
openai.api_key = os.getenv("OPENAI_API_KEY")

with open('all_scores.json', 'r') as file:
    data = json.load(file)

## GPT functions to retrieve text and image based on all_scores.json

In [ ]:
def generate_text_GPT(text_prompt):
    """
    Creates a new ChatGPT output, based on certain rules defined in the global variable "standard_rules".
    It clears the conversation history and add the new list items to it (starting a new chat).

    Input
    A text prompt that is meant for ChatGPT in string

    Returns
    ChatGPT output in a string

    """
    messages = [{"role": "system", "content": "Create the output without any markdown or markup"}]
    messages.append({"role": "user", "content": text_prompt})

    new = client.chat.completions.create(
                model="gpt-4o",
                messages=messages
            )

    # only get the output message 
    output = new.choices[0].message.content

    return output

def generate_image_GPT(text_prompt):
    """
    Creates a new DALLE-3 output, based on certain rules defined in the global variable "standard_rules".

    Input
    A text prompt that is meant for ChatGPT in string

    Returns
    Image URL (online for an hour)

    """

    response = client.images.generate(
                model="dall-e-3",
                prompt=text_prompt,
                size="1024x1024",
                quality="standard",
                n=1
            )

    # get image URL
    image_url = response.data[0].url

    return image_url


def output(data):
    amount_stages = data['stages']
    for i in range(amount_stages):
        i += 1
        print(f"log - stage: {i}")

        prompts = data[f'stage_{i}']['prompts']['text_prompt']
        for cat, prompt in prompts.items():
            # textual output
            print(f"log - category: {cat}, creating text")
            text_output = generate_text_GPT(prompt)
            data[f'stage_{i}']['output_text'][f'{cat}'] = text_output
            
            # sentiment analysis
            print('log - creating sentiment')
            blob = TextBlob(text_output, analyzer=NaiveBayesAnalyzer())
            data[f'stage_{i}']['sentiment'][f'{cat}'] = blob.sentiment

        prompts = data[f'stage_{i}']['prompts']['image_prompt']
        for cat, prompt in prompts.items():
            # image output
            print(f"log - category: {cat}, creating image")
            image_url = generate_image_GPT(prompt)
            image_output = requests.get(image_url).content
            with open(f"images2/S{i}{cat}.png", "wb") as file:
                file.write(image_output)


In [134]:
# Run functions
output(data)

log - stage: 1
log - category: B, creating text
log - creating sentiment
log - category: PHA, creating text
log - creating sentiment
log - category: NHA, creating text
log - creating sentiment
log - category: PLA, creating text
log - creating sentiment
log - category: NLA, creating text
log - creating sentiment
log - category: B, creating image
log - category: PHA, creating image
log - category: NHA, creating image
log - category: PLA, creating image
log - category: NLA, creating image
log - stage: 2
log - category: B, creating text
log - creating sentiment
log - category: PHA, creating text
log - creating sentiment
log - category: NHA, creating text
log - creating sentiment
log - category: PLA, creating text
log - creating sentiment
log - category: NLA, creating text
log - creating sentiment
log - category: B, creating image
log - category: PHA, creating image
log - category: NHA, creating image
log - category: PLA, creating image
log - category: NLA, creating image
log - stage: 3
log